# The Main Data Loading Pipeline Summarized
The data loading pipeline without the intermediate steps.

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.2.2
tiktoken version: 0.11.0


In [3]:
import os
import urllib.request
file_path = "../../data/the-verdict.txt"
if not os.path.exists(file_path):
  url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
  urllib.request.urlretrieve(url, file_path)

with open(file_path, "r", encoding="utf-8") as f:
  raw_text = f.read()  
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size, max_length, stride,shuffle=True, drop_last=True, num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

vocab_size = 50257
output_dim = 256
context_length = 1024


token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

batch_size = 8
max_length = 4
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length
)

In [6]:
for batch in dataloader:
    x, y = batch
    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))
    input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [7]:
print(input_embeddings)

tensor([[[ 1.2495, -1.7590,  0.3354,  ...,  0.0716, -1.0370,  0.4530],
         [ 0.6774, -1.2482,  0.2193,  ..., -1.3630, -2.1008, -0.2072],
         [-0.0491,  0.8185,  1.4194,  ..., -1.1851,  1.2192,  0.5295],
         [ 1.0044, -0.2523,  1.0076,  ...,  0.3774,  0.0793, -1.1029]],

        [[ 1.3737,  0.3177,  0.9934,  ...,  1.1267,  0.4131,  1.0237],
         [ 1.7872,  0.3709,  0.6397,  ...,  1.4415, -1.3131,  0.1603],
         [-0.1593, -2.5614,  1.9587,  ..., -2.4815, -0.3417, -0.4968],
         [-1.1420,  0.2221,  2.3220,  ...,  0.7290, -3.6830,  0.7922]],

        [[ 0.4895, -3.5707,  1.1923,  ...,  1.1356,  0.9608,  0.6742],
         [ 0.0425, -0.3738,  1.2895,  ...,  2.8365,  0.0060,  0.4089],
         [-0.2814,  0.2520,  1.4389,  ..., -2.1764,  0.6841,  1.1700],
         [ 1.6723,  0.4500, -0.7053,  ...,  1.3974, -2.8092,  0.2660]],

        ...,

        [[-0.5583, -0.4231,  1.0300,  ...,  0.3199, -0.1897,  0.9261],
         [ 0.9241, -0.8884,  1.4612,  ...,  0.7337, -0.64